<a href="https://colab.research.google.com/github/maikenberthelsen/Proj1/blob/master/unetup2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from __future__ import print_function


from google.colab import drive
drive.mount('/content/drive/')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import shutil
import sys
import urllib
import numpy as np
from PIL import Image
from pathlib import Path
import tensorflow.python.platform
import tensorflow as tf
from scipy import misc, ndimage
import shutil
from sklearn.utils import class_weight, shuffle


from mask_to_submission import *
from helpers import *
from image_processing import *
from image_augmentation import *
from F1_metrics import *
from data_extraction import *
from prediction import *
from unet_pred import *
from unetModel import *


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


NUM_CHANNELS = 3 # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 100
TESTING_SIZE = 50
VALIDATION_SIZE = 0  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16 # 64
NUM_EPOCHS = 5
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 1000
MAX_AUG = 6
NEW_DIM_TRAIN = 400

# The size of the patches each image is split into. Should be a multiple of 4, and the image
# size would be a multiple of this. For this assignment to get the delivery correct it has to be 16
IMG_PATCH_SIZE = 16
INPUT_CHANNELS = 3

# Extract data into numpy arrays, divided into patches of 16x16
data_dir = '/content/drive/My Drive/data/'
train_data_filename = data_dir + 'training/images/'
train_labels_filename = data_dir + 'training/groundtruth/' 
test_data_filename = data_dir + 'test_set_images'

# Directive for storing the augmented training images
imgDir = 'training/augmented/images'
groundTruthDir = 'training/augmented/groundtruth'



#earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.1)






Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [31]:
#x_train, y_train, x_test = load_data_img(train_data_filename, train_labels_filename, test_data_filename, TRAINING_SIZE, TESTING_SIZE, NEW_DIM_TRAIN)
#x_train, y_train, x_test = load_data_unet(train_data_filename, train_labels_filename, test_data_filename, TRAINING_SIZE, TESTING_SIZE, NEW_DIM_TRAIN)

x_train, y_train, x_test, x_val, y_val = load_data_unet(train_data_filename, train_labels_filename, test_data_filename, TRAINING_SIZE, TESTING_SIZE,VALIDATION_SIZE, NEW_DIM_TRAIN,
  saltpepper = 0.0,augment=True, MAX_AUG=MAX_AUG, augImgDir=imgDir , data_dir=data_dir, groundTruthDir =groundTruthDir)
print(y_train.shape)
print(x_train.shape)

Augmenting training images...
Directory  training/augmented/images  already exists, overwritten
Directory  training/augmented/groundtruth  already exists, overwritten
Test data shape:  (50, 608, 608, 3)
Number of samples in class 1 (background):  86706976
Number of samples in class 2 (road):  25293024 

(700, 400, 400, 2)
(700, 400, 400, 3)


In [32]:
# Class weigths
classes = np.array([0,1])
class_weights = class_weight.compute_class_weight('balanced',classes,y_train[:,:,:,0].flatten())
print('Class weights: ',class_weights) 
'''
yweight = y_train[:,:,:,0]
yweight = yweight.flatten()
print(np.unique(yweight), sum(yweight))
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(yweight),
                                                 yweight)'''

print('Class weights: ',class_weights) 

[0. 1.] 25293024.0
Class weights:  [0.64585345 2.21404922]


In [33]:
#model = ZF_UNET_224(class_weights,NEW_DIM_TRAIN)
inputs = Input((NEW_DIM_TRAIN, NEW_DIM_TRAIN,INPUT_CHANNELS))
model = create_model(inputs,n_filters=16, dropout=0.05)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 400, 400, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 400, 400, 16) 64          conv2d_20[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_19 (LeakyReLU)      (None, 400, 400, 16) 0           batch_normalization_19[0][0]     
__________________________________________________________________________________________________
conv2d_21 

In [0]:

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
# Checkpoint
filepath="/content/drive/My Drive/weightsUnet/weights.best.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True,mode='max')
]

In [36]:
# Train the model
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

X (700, 400, 400, 3) y (700, 400, 400, 2)
Train on 630 samples, validate on 70 samples
Epoch 1/5
320/630 [==============>...............] - ETA: 28s - loss: 0.6179 - acc: 0.6688

KeyboardInterrupt: ignored

In [0]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

In [0]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

In [0]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

In [0]:
'''print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          #validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )'''

In [0]:
'''print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          #validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )'''

In [0]:
'''print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          #validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )'''

In [0]:
'''print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          #validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )'''

In [0]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

In [0]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

In [0]:

list_filename = []
prediction_test_dir = "/content/drive/My Drive/predictions_test/"
GT_pred_test_dir = "/content/drive/My Drive/predictions_ground_test/"

if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)
if not os.path.isdir(GT_pred_test_dir):
    os.mkdir(GT_pred_test_dir)
    
y_submit = np.zeros((((608//IMG_PATCH_SIZE)**2)*TESTING_SIZE,2))
for i in range(1,TESTING_SIZE+1):
  gt_pred, orImg = get_pred_and_ysubmit_pixelwise(test_data_filename, i, 'test', model, PIXEL_DEPTH, NEW_DIM_TRAIN,IMG_PATCH_SIZE,prediction_test_dir)
  gt_filename = GT_pred_test_dir + "gt_pred_" + str(i) + ".png"
  list_filename.append(gt_filename)
  gt_pred.save(gt_filename)
  overlay2 = make_img_overlay_pixel(orImg, gt_pred, PIXEL_DEPTH)
  overlay2.save(GT_pred_test_dir + "overlay_" + str(i) + ".png")
  
  ## FOR OWN VALIDATION ONLY, NEED IT TO COUNT WHITE PATCHES
  gtarr = np.asarray(gt_pred)
  label_patches = img_crop(gtarr, IMG_PATCH_SIZE, IMG_PATCH_SIZE)
  data = np.asarray(label_patches)
  labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])
  newPred = label_to_img_unet(gtarr.shape[0], gtarr.shape[1],IMG_PATCH_SIZE, IMG_PATCH_SIZE, gtarr,'test')
  img = Image.fromarray(newPred)
  img.save(prediction_test_dir + "patch_gtimg_" + str(i) + ".png")
  y_submit[((608//IMG_PATCH_SIZE)**2)*(i-1):((608//IMG_PATCH_SIZE)**2)*i,:] = labels
  overlay = make_img_overlay_pixel(orImg, img, PIXEL_DEPTH)
  overlay.save(prediction_test_dir + "overlay_" + str(i) + ".png")
  
  
print('y_submit: ', y_submit.shape)
print('antall vei / antall bakgrunn: ', np.sum(y_submit[:,1]))

In [0]:
masks_to_submission("kerasMask.csv", *list_filename)


In [0]:
prediction_training_dir = "/content/drive/My Drive/predictions_training/"

if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, TRAINING_SIZE+1):
    oimg, imgpred = get_prediction_with_overlay_pixelwise(train_data_filename, i, 'train', model, PIXEL_DEPTH, NEW_DIM_TRAIN,IMG_PATCH_SIZE)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")
    imgpred.save(prediction_training_dir + "predictimg_" + str(i) + ".png")


In [0]:
# UNNECESSARY
# Make submission file
prediction_to_submission2('submission_keras.csv', y_submit)

**POST-PROCESSING**

In [0]:
new_test_filename = '/content/drive/My Drive/test_set_post_images/'
post_processed_list = []
if not os.path.isdir(new_test_filename):
    os.mkdir(new_test_filename)

#y_submit_post = np.zeros((((608//IMG_PATCH_SIZE)**2)*TESTING_SIZE,2))
for i in range(1,TESTING_SIZE+1):
    p_img = get_postprocessed_unet(GT_pred_test_dir, i, 'test')
    filename = new_test_filename + "processedimg_" + str(i) + ".png"
    post_processed_list.append(filename)
    p_img.save(filename)
    pred = Image.open(filename)
    pred = pred.convert('RGB')
    imageid = "/test_%d" % i
    image_filename = test_data_filename + imageid + imageid + ".png"
    overlay = make_img_overlay_pixel(orImg, pred, PIXEL_DEPTH)
    overlay.save(new_test_filename + "overlay_" + str(i) + ".png")

masks_to_submission("kerasPostprocessedMask.csv", *post_processed_list)

In [0]:
  y_pred_train = np.zeros((((400//IMG_PATCH_SIZE)**2)*TRAINING_SIZE,2))
  y_val_train = np.zeros((((400//IMG_PATCH_SIZE)**2)*TRAINING_SIZE,2))
  for i in range(VALIDATION_SIZE):
    x_img = x_val[i,:,:,:]
    x_img = Image.fromarray(x_img)
    output_prediction = get_prediction_pixel(x_img, model, NEW_DIM_TRAIN) #(1,224,224)
    output_prediction = np.transpose(output_prediction, (1, 2, 0)) #(224,224,1)
    predict_img = np.asarray(output_prediction)

    # Changes into a 3D array, to easier turn into image
    predict_img_3c = np.zeros((predict_img.shape[0],predict_img.shape[1], 3), dtype=np.uint8)
    predict_img8 = np.squeeze(img_float_to_uint8(predict_img, PIXEL_DEPTH))
    predict_img8[predict_img8 >= 100] = 255 
    predict_img8[predict_img8 < 100] = 0        
    predict_img_3c[:,:,0] = predict_img8
    predict_img_3c[:,:,1] = predict_img8
    predict_img_3c[:,:,2] = predict_img8
    #imgpred = Image.fromarray(predict_img_3c)
    label_patches = img_crop(predict_img_3c, IMG_PATCH_SIZE, IMG_PATCH_SIZE)
    data = np.asarray(label_patches)
    labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])
    #newPred = label_to_img_unet(gtarr.shape[0], gtarr.shape[1],IMG_PATCH_SIZE, IMG_PATCH_SIZE, gtarr,'test')
    #img = Image.fromarray(newPred)
    #img.save(prediction_test_dir + "patch_gtimg_" + str(i) + ".png")
    y_pred_train[((400//IMG_PATCH_SIZE)**2)*(i):((400//IMG_PATCH_SIZE)**2)*(i+1),:] = labels
    print(labels.shape)
    y_label_patches = img_crop(y_val[i,:,:,:], IMG_PATCH_SIZE, IMG_PATCH_SIZE)
    y_data = np.asarray(y_label_patches)
    y_labels = np.asarray([value_to_class(np.mean(y_data[i])) for i in range(len(y_data))])
    y_val_train[((400//IMG_PATCH_SIZE)**2)*(i):((400//IMG_PATCH_SIZE)**2)*(i+1),:] = y_labels

  
  
tp, tn, fp, fn = f1_values(y_pred_train, y_val_train[:,1])
f1 = f1_measure(tp, fp, fn)
print("f1", f1)

In [0]:
for i in range(1,TESTING_SIZE+1):
    #test_data_filename = data_dir + 'test_set_images'
    
    #oimg, gtimg = get_prediction_with_overlay_pixelwise(test_data_filename, i, 'test', model, PIXEL_DEPTH, NEW_DIM_TRAIN)
    #oimg.save(prediction_test_dir + "overlay_" + str(i) + ".png")
    y_submit_post[((608//IMG_PATCH_SIZE)**2)*(i-1):((608//IMG_PATCH_SIZE)**2)*i,:], p_img = get_pred_postprocessed_unet(prediction_test_dir, i, 'test',IMG_PATCH_SIZE)
    filename = new_test_filename + "processedimg_" + str(i) + ".png"
    p_img.save(filename)
    pred = Image.open(filename)
    pred = pred.convert('RGB')
    imageid = "/test_%d" % i
    image_filename = test_data_filename + imageid + imageid + ".png"
    orImg = Image.open(image_filename)
    #img.save(prediction_test_dir + "patch_gtimg_" + str(i) + ".png")
    print(np.asarray(pred).shape)
    print(np.asarray(p_img).shape)
    #y_submit[((608//IMG_PATCH_SIZE)**2)*(i-1):((608//IMG_PATCH_SIZE)**2)*i,:] = labels
    overlay = make_img_overlay_pixel(orImg, pred, PIXEL_DEPTH)
    overlay.save(new_test_filename + "overlay_" + str(i) + ".png")